# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import copy
import pickle

import torch
from torch_geometric.data import Data

import wandb

import sbmlcore
import sbmlsim
import gumpy

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
full_dataset = torch.load('full_orig_dataset.pth')

### Set Up Params and Sweep Config

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = 10000
cutoff_distance = 6.3  

# gcn params
ews = 'none'
num_node_features = 12
batch_size = 256
hidden_channels = 64
learning_rate = 0.001
wd = 5e-5
epochs = 400

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [4]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    # 'optimizer': {
    #     'values': ['adam', 'sgd']
    #     },
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

sweep_config['parameters'] = parameters_dict

# parameters_dict.update({
#     'epochs': {
#         'value': 400}
#     })

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-6,
        'max': 1e-1
      },
    # 'batch_size': {
    #     # integers between 32 and 256
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform_values',
    #     'q': 8,
    #     'min': 32,
    #     'max': 256,
    #   }
    })



In [5]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'hidden_channels': {'values': [32, 64, 128, 256]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.1,
                                  'min': 1e-06}}}


### Define Training Loop

In [6]:
sweep_id = wandb.sweep(sweep_config, project="pnca-sweep-1")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Create sweep with ID: 93peolwf
Sweep URL: https://wandb.ai/dylan-home/pytorch-sweeps-demo/sweeps/93peolwf


In [10]:
def sweep_run():

    with wandb.init() as run:
        config = run.config
        
        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = cutoff_distance,
            # edge_weight_func = '1-(dist/cutoff)',
            edge_weight_func= 'none',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = wd,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        torch.save(model, f'saved_models/carter_ds_aug/sweep/{datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")}')

In [12]:
wandb.agent(sweep_id, sweep_run, count=30)

wandb: Agent Starting Run: sdll7idk with config:
wandb: 	hidden_channels: 32
wandb: 	learning_rate: 0.010070681388818925
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6916, Test Loss: 0.6931
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6918, Test Loss: 0.6933
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6921, Test Loss: 0.6937
Epoch: 040, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6919, Test Loss: 0.6931
Epoch: 050, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6919, Test Loss: 0.6938
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


Test Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,▃▃▂▃▁▅▃▆▃▇▂▃▂▂▂▃▃▄▅▂▂▃▄▄▁█▂▂▆▃▂▂▂▃▁▆▁▄▂▁
Test Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,▅▂▂▁▄▃▂▃▃▄▂▁▄▅▂▃▂▂▂▂▂▅▄▅▆█▃▄▂▂▃▃▃▁▄▃▄▄▂▃
Train Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,0.511


wandb: Agent Starting Run: arft566f with config:
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 6.063489646943198e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.7027, Test Loss: 0.7102
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6961, Test Loss: 0.7015
Epoch: 030, Train Acc: 0.5256, Test Acc: 0.5077, Train Loss: 0.6940, Test Loss: 0.6976
Epoch: 040, Train Acc: 0.4766, Test Acc: 0.4070, Train Loss: 0.6943, Test Loss: 0.6971
Epoch: 050, Train Acc: 0.5064, Test Acc: 0.4600, Train Loss: 0.6939, Test Loss: 0.6970
Epoch: 060, Train Acc: 0.5260, Test Acc: 0.5090, Train Loss: 0.6932, Test Loss: 0.6974
Epoch: 070, Train Acc: 0.5171, Test Acc: 0.4840, Train Loss: 0.6932, Test Loss: 0.6963
Epoch: 080, Train Acc: 0.5264, Test Acc: 0.5103, Train Loss: 0.6930, Test Loss: 0.6967
Epoch: 090, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6927, Test Loss: 0.6968
Epoch: 100, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6930, Test Loss: 0.6967
Epoch: 110, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6923

...

Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.4736, Test Acc: 0.4890, Train Loss: 0.6934, Test Loss: 0.6933
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6917, Test Loss: 0.6935
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6921, Test Loss: 0.6929
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


Test Accuracy,█████████▁█████▁█████████████████████
Test F1,█████████▁█████▁█████████████████████
Test Loss,▂▁▁▁▃▂▂▁▂▁▁█▃▁▁▁▁▁▁▁▁▁▁▂▂▂▁▃▁▁▂▁▁▁▁▂▁
Test Sensitivity,█████████▁█████▁█████████████████████
Test Specificity,▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,█████████▁█████▁█████████████████████
Train F1,█████████▁█████▁█████████████████████
Train Loss,▂▁▂▂▃▁▁▁▂▂▁█▃▂▂▂▂▁▂▁▁▂▁▁▁▁▂▂▁▁▁▁▂▂▁▁▁
Train Sensitivity,█████████▁█████▁█████████████████████
Train Specificity,▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,0.511


wandb: Agent Starting Run: fkyiu579 with config:
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 4.168738429022371e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5026, Test Acc: 0.4503, Train Loss: 0.6937, Test Loss: 0.6964
Epoch: 020, Train Acc: 0.5223, Test Acc: 0.5030, Train Loss: 0.6921, Test Loss: 0.6941
Epoch: 030, Train Acc: 0.4773, Test Acc: 0.4933, Train Loss: 0.6942, Test Loss: 0.6934
Epoch: 040, Train Acc: 0.6253, Test Acc: 0.6330, Train Loss: 0.6897, Test Loss: 0.6890
Epoch: 050, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.7028, Test Loss: 0.7081
Epoch: 060, Train Acc: 0.5253, Test Acc: 0.5103, Train Loss: 0.6841, Test Loss: 0.6846
Epoch: 070, Train Acc: 0.5264, Test Acc: 0.5103, Train Loss: 0.6840, Test Loss: 0.6852
Epoch: 080, Train Acc: 0.5266, Test Acc: 0.5110, Train Loss: 0.6841, Test Loss: 0.6855
Epoch: 090, Train Acc: 0.5146, Test Acc: 0.4950, Train Loss: 0.6784, Test Loss: 0.6772
Epoch: 100, Train Acc: 0.6640, Test Acc: 0.6590, Train Loss: 0.6778, Test Loss: 0.6726
Epoch: 110, Train Acc: 0.4983, Test Acc: 0.4730, Train Loss: 0.6741

Test Accuracy,▂▂▂▂▄▂▂▂▅▂▂▂▆▆▁▁▁▆▁▁▁▇▁▇▇▇▇▇▆▇▆▂▇▆▇▇▇███
Test F1,▇▇▇▇▁▇▇▇▃▇▇▇▅▆▆▇▆▅▇▆▇▇▆▆▇▆▆▇▆▆▅▇▇▅▆██▇██
Test Loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▄▄▂▄▂▂▃▁▁▁
Test Sensitivity,████▁███▂███▃▄▆▇▇▃▇▇▇▄▆▄▅▄▄▅▃▄▃▇▄▃▃▅▆▅▅▅
Test Specificity,▁▁▁▁█▁▁▁█▁▁▁█▇▂▁▂▇▁▂▁▇▂▇▆▇▇▇▇▇▇▂▇█▇▆▅▇▇▇
Train Accuracy,▂▂▂▂▂▂▂▂▂▁▂▁▃▅▁▁▁▄▁▁▁▆▁▆▇▆▆▇▆▆▅▂▇▄▆█████
Train F1,▇▇▇▇▁▇▇▇▂▇▇▇▃▆▆▇▇▄▇▇▇▆▆▆▇▆▆▇▆▆▅▇▇▄▅██▇██
Train Loss,███▇█▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▆▅▅▅▄▄▄▄▄▄▅▄▃▅▄▂▂▂▁▁
Train Sensitivity,████▁███▁███▂▄▇▇▇▃▇▇▇▄▆▄▅▄▄▅▃▄▃▇▄▃▃▆▇▅▅▅
Train Specificity,▁▁▁▁█▁▁▁█▁▁▁█▇▂▁▂█▂▂▁▇▂▇▆▇▇▇███▂▇██▇▆▇▇▇
Test Accuracy,0.72333
